# DSL development

## Imports

In [ ]:
# Use this to reload changes in python scripts
%reload_ext autoreload
%autoreload 1
%aimport arc25.dsl, arc25.training_tasks, arc25.input_generation
%aimport -matplotlib, -matplotlib.pyplot

In [ ]:
import random
import logging
import numpy as np
from IPython.display import Markdown, display
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm.auto import tqdm

from arc25.dsl import *
from arc25.training_tasks import *
from arc25.input_generation import *
from arc25.plot import plot_task, plot_grid, plot_grids_with_shape
from arc25.logging import configure_logging
from arc25.code_analysis import analyze_dsl_usage

#configure_logging(level=logging.DEBUG)
configure_logging(level=logging.INFO)
logging.getLogger('matplotlib').setLevel(logging.CRITICAL)


plt.plot()
plt.close('all')
plt.rcParams["figure.figsize"] = (20, 3)  
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['font.size'] = 12

## Individual tasks

- Color should be different when touching other objects
- When monochrome there can't be contact

In [ ]:
task_generator = DrawPixelsWithCentersReferencePointsColorBasedOnArea()
for _ in range(3):
    task = task_generator.sample()
    display(Markdown("```python\n" + task.code + "\n```"))
    plot_task(task); plt.show()

In [ ]:
[task_generator.sample() for _ in tqdm(range(100), smoothing=0)];

## All tasks analysis

In [ ]:
generator = training_tasks_generator()
tasks = [next(generator) for _ in tqdm(range(100), smoothing=0)];

In [ ]:
analyze_dsl_usage([task.code for task in tasks])

## Visualize generators

In [ ]:
imgs = []
for _ in range(3):
    imgs.append(generate_arc_image_with_random_objects(
        image_shape=(10, 10),
        n_objects=5,
        allowed_sizes=(1, 3),
        monochrome=True,
        random_shape_probability=0.0,
        line_shape_probability=0.5,
    )[0])
plot_grids_with_shape(imgs)

In [ ]:
imgs = []
for _ in range(3):
    imgs.append(generate_arc_image_with_random_objects(
        image_shape=(10, 10),
        n_objects=2,
        allowed_sizes=[1],
        random_shape_probability=0.25,
        line_shape_probability=0.5,
    )[0])
plot_grids_with_shape(imgs)